# Prepare synthetic data (Vitals)
It is going to be much easier to develop if you have realistic synthetic data.
Here we take a sql query that generates a single tabular output.
We run that query against the live identifiable data once.
We then use the [Synthetic Data Vault](https://sdv.dev/SDV/index.html) to prepare a synthetic model of those data.
The code below serves as a vignette for that process but will need adjusting to match the exact contents of the original query.

More complex examples that include multiple tables with joins and dependencies are also possible.

This notebook should be run interactively just once

## Set-up, query and return the data as datafame
The query lives in `./src/api/sitrep/` where `.` represents the project root.
If you run this JupyterNotebook using the local **Makefile** and `make run` then that query will be automatically copied here.

So first steps should be (from _this_ directory)
```sh
make mock1build
make mock2copyin
make mock3run
```
then navigate to http://uclvlddpragae07:8091/lab/tree/steve/work/synth_test_data_vitals.ipynb

In [21]:
import os
import pandas as pd
import numpy as np
import requests

from pathlib import Path
import sqlalchemy as sa

In [22]:
# Construct the PostgreSQL connection
emap_host = os.getenv('EMAP_DB_HOST')
emap_user = os.getenv('EMAP_DB_USER')
emap_passwd = os.getenv('EMAP_DB_PASSWORD')

emapdb_engine = sa.create_engine(f'postgresql://{emap_user}:{emap_passwd}@{emap_host}:5432/uds')

In [23]:
# Read the sql file into a query 'q' and the query into a dataframe
q = Path('vitals.sql').read_text()

In [24]:
df = pd.read_sql_query(sa.text(q), emapdb_engine)

Now rather than using a SQL query, this time we are going to use an existing API to populate a dataframe

In [25]:
df.shape

(4015, 16)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4015 entries, 0 to 4014
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   visit_observation_id  4015 non-null   int64              
 1   ob_tail_i             4015 non-null   int64              
 2   observation_datetime  4015 non-null   datetime64[ns, UTC]
 3   id_in_application     4015 non-null   object             
 4   value_as_real         2101 non-null   float64            
 5   value_as_text         1913 non-null   object             
 6   unit                  526 non-null    object             
 7   mrn                   4015 non-null   object             
 8   lastname              4015 non-null   object             
 9   firstname             4015 non-null   object             
 10  sex                   4015 non-null   object             
 11  date_of_birth         4015 non-null   object             
 12  bed_ad

## Generate a synthetic version of the real data

Use the table above to generate the metadata you need for the synthetic data

https://sdv.dev/SDV/user_guides/relational/relational_metadata.html#relational-metadata

In [27]:
# Minimal imports
from sdv import Metadata, SDV
from sdv.constraints import FixedCombinations

SDV does not handle timezones nicely so remove

In [28]:
# PostgreSQL returns datetimes with tz info which sdv does not seem to be able to handle
def remove_timezone(df, col_name: str) -> pd.DataFrame:
    """sdv does not like timezones"""
    df[col_name] = pd.to_datetime(df[col_name], utc=True).dt.tz_localize(None)    
    return df

In [29]:
tz_cols = ['observation_datetime', 'bed_admit_dt']
for col in tz_cols:
    print(col)
    df = remove_timezone(df, col)

observation_datetime
bed_admit_dt


### Define PII that must be faked and not modelled

Define fields that contain PII and need faking (see the sketchy documentation [here](https://sdv.dev/SDV/developer_guides/sdv/metadata.html?highlight=pii#categorical-fields-data-anonymization) and the [Faker Documentation](https://faker.readthedocs.io/en/master/providers.html) for a full list of providers. Here is a brief example that specifies Fakers for [name](https://faker.readthedocs.io/en/master/providers/faker.providers.person.html#faker.providers.person.Provider.name) and [date of birth](https://faker.readthedocs.io/en/master/providers/faker.providers.date_time.html#faker.providers.date_time.Provider.date_of_birth). Note that you must pass arguments to a faker as a list.

NB: sdv also doesn't always recognise the columns correctly. Here we specify data_of_birth explicitly as a date whilst working on the larger task of defining columns that contain PII. See [field details](https://sdv.dev/SDV/developer_guides/sdv/metadata.html#field-details)

Now a full specification for the Vitals data

In [30]:
fields = {
    'visit_observation_id' : {
        'type': 'id',
        'subtype': 'integer',
    },
    'date_of_birth': {
        'type': 'datetime',
        'format': '%Y-%m-%d',
        'pii': True,
        # the 'pii_category' key defines the Faker function name (method)
        'pii_category': "date_of_birth", 
    },
    'lastname': {
        'type': 'categorical',
        'pii': True,
        'pii_category': 'last_name'
    },
    'firstname': {
        'type': 'categorical',
        'pii': True,
        'pii_category': 'first_name'
    },
    'mrn': {
        'type': 'categorical',
        'pii': True,
        'pii_category': ['random_number', 8 ]
    },
}

Prepare the constraints

In [40]:
fixed_obs_constraint = FixedCombinations(
    column_names=['id_in_application', 'value_as_real', 'value_as_text', 'unit'],
    handling_strategy='reject_sampling',  # transform strategy fails ?b/c data types vary
)

Prepare the metadata

In [41]:
metadata = Metadata()
metadata.add_table(
    name='tabpid',
    data=df,
    fields_metadata=fields,
    primary_key = 'visit_observation_id',
)

In [42]:
# Inspect the conversion that metadata.add_table did to the dataframe that you loaded
metadata.get_table_meta('tabpid')

{'fields': {'visit_observation_id': {'type': 'id', 'subtype': 'integer'},
  'date_of_birth': {'type': 'datetime',
   'format': '%Y-%m-%d',
   'pii': True,
   'pii_category': 'date_of_birth'},
  'lastname': {'type': 'categorical',
   'pii': True,
   'pii_category': 'last_name'},
  'firstname': {'type': 'categorical',
   'pii': True,
   'pii_category': 'first_name'},
  'mrn': {'type': 'categorical',
   'pii': True,
   'pii_category': ['random_number', 8]},
  'ob_tail_i': {'type': 'numerical', 'subtype': 'integer'},
  'observation_datetime': {'type': 'datetime'},
  'id_in_application': {'type': 'categorical'},
  'value_as_real': {'type': 'numerical', 'subtype': 'float'},
  'value_as_text': {'type': 'categorical'},
  'unit': {'type': 'categorical'},
  'sex': {'type': 'categorical'},
  'bed_admit_dt': {'type': 'datetime'},
  'dept_name': {'type': 'categorical'},
  'room_name': {'type': 'categorical'},
  'bed_hl7': {'type': 'categorical'}},
 'primary_key': 'visit_observation_id'}

Prepare the table(s)

In [43]:
tables = dict(tabpid=df)

Fit the model

In [44]:
from sdv.lite import TabularPreset
model = TabularPreset(name='FAST_ML', 
                      metadata=metadata.get_table_meta('tabpid'), 
                      constraints=[fixed_obs_constraint], # be patient; reject_sampling is slow
)

model.fit(df)
dfs=model.sample(df.shape[0])

In [45]:
dfs.loc[:30]

,visit_observation_id,ob_tail_i,observation_datetime,id_in_application,value_as_real,value_as_text,unit,mrn,lastname,firstname,sex,date_of_birth,bed_admit_dt,dept_name,room_name,bed_hl7
0,6,1,2022-06-26 07:48:35.916419840,28315,NaN,2,NaN,9080689,Cook,Ernest,M,2016-04-06,2022-06-27 03:22:36.093017856,UCH T14 SOUTH ASU,POOL ROOM,13-NON COVID MAJORS
1,9,1,2022-06-26 10:12:10.332110848,28315,NaN,3,NaN,46346917,Castillo,Yvonne,M,1972-12-03,2022-05-26 07:52:49.299431424,UCH T09 CENTRAL (T09C),BY10,SR03-03
2,10,1,2022-06-26 06:30:46.844068608,6466,NaN,A,NaN,34245512,Hicks,Mathew,F,1950-09-28,2022-06-21 19:17:31.564380672,UCH T07 NORTH (T07N),UTC POST TRIAGE WAITING ROOM,BY02-05
3,15,1,2022-06-26 09:10:20.023743488,5,NaN,110/79,NaN,73614945,Smith,Laura,F,1985-09-16,2022-06-04 23:16:04.640171520,UCH T10 NORTH (T10N),SR09,SR34-34
4,31,1,2022-06-26 08:29:27.773159936,3040109304,NaN,Room air,NaN,23361993,Hansen,Tiffany,F,1912-12-28,2022-06-20 17:29:08.986451200,UCH T03 INTENSIVE CARE,BY12,SR50-50
5,50,1,2022-06-26 09:47:37.658296064,28315,NaN,1,NaN,92669582,Williams,Alicia,M,1985-05-08,2022-05-26 00:54:49.337675776,UCH T03 INTENSIVE CARE,SR06,BY12-58
6,56,1,2022-06-26 07:58:58.798527488,28315,NaN,NaN,NaN,36023994,Gibbs,Alexis,M,1989-03-20,2022-07-02 09:45:01.719075840,UCH T09 SOUTH (T09S),BY04,BY07-42
7,60,1,2022-06-26 07:06:22.610288384,3040109304,NaN,Room air,NaN,89757448,Brown,Donald,M,2010-11-06,2022-05-31 12:04:54.002069504,UCH T07 NORTH (T07N),BY02,BY10-48
8,65,1,2022-06-26 08:59:54.371040000,28315,NaN,1,NaN,60771832,Wagner,Maria,F,2015-10-24,2022-05-18 13:38:45.767711744,UCH T13 SOUTH (T13S),BY11,SR09-09
9,84,1,2022-06-26 09:49:57.182713856,9,10.0,NaN,NaN,27944313,Meyer,Tracy,F,1996-03-18,2022-06-26 23:21:02.268831232,UCH T03 INTENSIVE CARE,NON COVID MAJORS 14,SR10-10


Inspect the original data

In [17]:
# df.head()

Inspect the synthetic data

Finally transform admission_age_years to match the fake DoB

### Save the synthetic data

Options
- save the model and not the synthetic data (but then you need *sdv* to run the model)
- save the data (need some care with type conversions if you use csv etc.)

In [46]:
model.save('mock_vitals.pkl')

In [47]:
dfs.to_hdf('mock_vitals.h5', 'data')

/tmp/ipykernel_291/1530517875.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block3_values] [items->Index(['id_in_application', 'value_as_text', 'unit', 'mrn', 'lastname',
       'firstname', 'sex', 'date_of_birth', 'dept_name', 'room_name',
       'bed_hl7'],
      dtype='object')]

  dfs.to_hdf('mock_vitals.h5', 'data')


In [48]:
pd.read_hdf('mock_vitals.h5', 'data')

,visit_observation_id,ob_tail_i,observation_datetime,id_in_application,value_as_real,value_as_text,unit,mrn,lastname,firstname,sex,date_of_birth,bed_admit_dt,dept_name,room_name,bed_hl7
0,6,1,2022-06-26 07:48:35.916419840,28315,NaN,2,NaN,9080689,Cook,Ernest,M,2016-04-06,2022-06-27 03:22:36.093017856,UCH T14 SOUTH ASU,POOL ROOM,13-NON COVID MAJORS
1,9,1,2022-06-26 10:12:10.332110848,28315,NaN,3,NaN,46346917,Castillo,Yvonne,M,1972-12-03,2022-05-26 07:52:49.299431424,UCH T09 CENTRAL (T09C),BY10,SR03-03
2,10,1,2022-06-26 06:30:46.844068608,6466,NaN,A,NaN,34245512,Hicks,Mathew,F,1950-09-28,2022-06-21 19:17:31.564380672,UCH T07 NORTH (T07N),UTC POST TRIAGE WAITING ROOM,BY02-05
3,15,1,2022-06-26 09:10:20.023743488,5,NaN,110/79,NaN,73614945,Smith,Laura,F,1985-09-16,2022-06-04 23:16:04.640171520,UCH T10 NORTH (T10N),SR09,SR34-34
4,31,1,2022-06-26 08:29:27.773159936,3040109304,NaN,Room air,NaN,23361993,Hansen,Tiffany,F,1912-12-28,2022-06-20 17:29:08.986451200,UCH T03 INTENSIVE CARE,BY12,SR50-50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4010,78,1,2022-06-26 08:05:47.130423552,6466,NaN,A,NaN,14171506,Dean,Thomas,M,2020-06-21,2022-07-19 01:23:47.298662144,UCH EMERGENCY DEPT,BY03,SR35-35
4011,82,1,2022-06-26 10:48:31.427398656,3040109304,NaN,Room air,NaN,15503770,Chang,Melinda,M,1975-08-02,2022-07-17 16:02:17.304674560,UCH T14 SOUTH ASU,BY01,BY04-21
4012,87,1,2022-06-26 08:24:50.558608128,28315,NaN,NaN,NaN,24410609,Hernandez,Alyssa,M,1965-08-18,2022-05-24 22:07:44.386354176,UCH EMERGENCY DEPT,UTC POST TRIAGE WAITING ROOM,SR06-06
4013,90,1,2022-06-26 08:55:01.091407104,6466,NaN,A,NaN,26679101,Caldwell,Sarah,F,1948-12-25,2022-06-28 16:55:03.905509376,UCH T08 SOUTH (T08S),BY05,SDEC OTF
